# Visualization notebook

This notebook is meant for visualizing stuff and testing code. 


## Imports and functions

In [1]:
import utils
import plotly.express as px
import numpy as np
import pandas as pd
import warnings
import spacy
import json
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

# MUD: 1,071,477 authors

### Plotting by bin

In [40]:
bin_nums = "1 2 3 4 5 6 7 8".split()

dev_bin_avg_docs = []




# manually recorded
HALF_FEATS_ACCS = [0.0, 0.02857142857142857, 0.08571428571428572, 0.11428571428571428, 0.22857142857142856, 0.22857142857142856, 0.22857142857142856, 0.4]
ALL_FEATS_ACCS = [0.0, 0.05714285714285714, 0.11428571428571428, 0.17142857142857143, 0.2, 0.2, 0.2857142857142857, 0.5428571428571428]


 

df = pd.DataFrame.from_dict({"half_features": HALF_FEATS_ACCS,
                             "all_features" : ALL_FEATS_ACCS,
                             "bin":bin_nums})


fig = px.line(df, x="bin", y=df.columns[0:2], template="plotly_dark", markers=True,)
fig.update_layout(title_text="Accuracy per bin", title_x=0.5)



In [6]:
from gram2vec.featurizers import GrammarVectorizer

g2v = GrammarVectorizer()

doc_vector = g2v.vectorize("Hello, this is a string 😀", return_vector=False)



['hello', 'this', 'is', 'a', 'string']